In [1]:
!pip -q install numpy pandas scikit-learn plotly

import numpy as np
import pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss
import plotly.graph_objects as go
import plotly.express as px

In [2]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


# Simulate Data

**What this section does:** This creates synthetic but realistic data to build and test the OCR optimization pipeline without needing real OCR logs.

**Key parameters to understand:**
- **TRUE_TAU = 80:** This represents the true threshold where OCR reliability changes from "often wrong" to "often right"
- **K = 5.0:** Controls the sharpness of the transition (smaller values = sharper change)
- **For each simulated item, we generate:**
  - A confidence score (0–99) from the OCR system
  - Ground truth correctness (is_correct) using a sigmoid curve where higher scores are more likely correct
  - A human reviewer (A/B/C) with varying reliability levels
  - A review label (the reviewer's judgment, which may be incorrect)
  - A timestamp for temporal ordering

**Output:** A dataset with columns: id, score, is_correct, reviewer_id, review_label, timestamp

This synthetic data serves as the training foundation for the rest of the analysis.


In [3]:
rng = np.random.default_rng(42)

n = 5000
TRUE_TAU = 80       # true threshold
K = 5.0             # slope of sigmoid (steepness)
REVIEWERS = {"A": 0.95, "B": 0.85, "C": 0.70}

def p_correct(score, tau=TRUE_TAU, k=K):
    """True underlying probability that OCR is correct."""
    return 1 / (1 + np.exp(-(score - tau) / k))

scores = rng.integers(0, 100, size=n)
probs = p_correct(scores)
is_correct = rng.binomial(1, probs)

reviewer_id = rng.choice(list(REVIEWERS.keys()), size=n)
review_label = np.array([
    (1 - is_correct[i]) if (rng.random() > REVIEWERS[r]) else is_correct[i]
    for i, r in enumerate(reviewer_id)
])

timestamp = pd.to_datetime("2025-08-01") + pd.to_timedelta(
    rng.integers(0, 60, size=n), unit="D"
)

df = pd.DataFrame({
    "id": np.arange(n),
    "score": scores,
    "is_correct": is_correct,
    "reviewer_id": reviewer_id,
    "review_label": review_label,
    "timestamp": timestamp
}).sort_values("timestamp").reset_index(drop=True)

df.head()

,id,score,is_correct,reviewer_id,review_label,timestamp
0,3948,96,1,C,0,2025-08-01
1,4541,62,0,B,1,2025-08-01
2,1183,44,0,A,0,2025-08-01
3,1564,55,0,B,0,2025-08-01
4,772,0,0,C,0,2025-08-01


# Exploratory Data Analysis (EDA)

**What this section does:** Performs basic sanity checks and creates intuition-building visualizations to understand the data patterns.

**Visualizations created:**
- **Score Distribution Histogram:** Shows the frequency of each confidence level in your dataset
- **Accuracy vs Score Bins:** Demonstrates how correctness increases with higher scores
- **True Threshold Marker:** A dashed line at TRUE_TAU=80 shows the intended reliability tipping point

**What to look for in the results:**
- A clear upward trend in the accuracy curve as scores increase
- Low scores should show mostly incorrect results
- High scores should show mostly correct results
- The curve should be smooth and monotonic

**Red flags to watch for:**
- Flat or decreasing accuracy curves (indicates data issues)
- Irregular patterns that don't match expected behavior
- If the true threshold marker doesn't align with the accuracy transition point


In [4]:
# Score distribution
fig1 = px.histogram(df, x="score", nbins=20, title="OCR Confidence Score Distribution",
                    color_discrete_sequence=["#007acc"])
fig1.show()

# Accuracy vs. score bins
bins = np.arange(0, 101, 5)
acc_by_bin = df.groupby(pd.cut(df["score"], bins), observed=False)["is_correct"].mean().reset_index()
acc_by_bin["score_bin"] = bins[:-1] + 2.5

fig2 = px.line(acc_by_bin, x="score_bin", y="is_correct", markers=True,
               title="Empirical Accuracy by Score Bin",
               labels={"is_correct": "Accuracy", "score_bin": "Score"})
fig2.add_vline(x=TRUE_TAU, line_dash="dash", line_color="red", annotation_text="True τ*=80")
fig2.show()

# Train/Validation Split

**What this section does:** Divides the dataset into training and validation sets using temporal ordering to simulate real-world deployment scenarios.

**Method:** 
- **Training set:** Uses the first 80% of data (chronologically earlier)
- **Validation set:** Uses the last 20% of data (chronologically later)

**Why temporal splitting matters:**
- **Realistic evaluation:** Mimics how you would deploy a model trained on historical data to make predictions on future data
- **Prevents data leakage:** Avoids the common mistake of using future information to predict the past
- **Tests model robustness:** Ensures your calibration works on new data, not just the data it was trained on

**Best practice:** Always use temporal splits for time-series data to get honest performance estimates.


In [9]:
cut = int(0.8 * len(df))
train = df.iloc[:cut].copy()
val   = df.iloc[cut:].copy()
print(f"Train size: {len(train)}, Validation size: {len(val)}")

Train size: 4000, Validation size: 1000


# Reviewer Reliability Analysis

**What this section does:** Estimates the trustworthiness of each human reviewer by analyzing their agreement with ground truth labels.

**Process:**
- **Agreement calculation:** For each reviewer, measures how often their review_label matches the true is_correct
- **Reliability scoring:** Converts agreement rates into reliability scores (0-1 scale) using Bayesian estimation
- **Weight assignment:** Uses these reliability scores as sample weights for model training

**Key insights:**
- **Reviewer A:** Highest reliability (0.95) - most trustworthy
- **Reviewer B:** Medium reliability (0.85) - moderately reliable  
- **Reviewer C:** Lower reliability (0.70) - least reliable

**Why this matters:**
- **Quality weighting:** More reliable reviewers have greater influence on model training
- **Noise reduction:** Prevents unreliable reviewers from misleading the calibration process
- **Realistic modeling:** Reflects that human reviewers vary in accuracy and consistency

**Visualization:** The bar chart shows each reviewer's reliability score, helping you identify which reviewers to trust most.


In [11]:
# Reliability (how often reviewer matches ground truth)
g = train.groupby("reviewer_id").apply(
    lambda d: pd.Series({
        "n": len(d),
        "agree": (d["review_label"] == d["is_correct"]).sum()
    })
)
g["alpha"] = 1 + g["agree"]
g["beta"]  = 1 + (g["n"] - g["agree"])
g["mean"]  = g["alpha"] / (g["alpha"] + g["beta"])

weights_map = g["mean"].to_dict()
train["w"] = train["reviewer_id"].map(weights_map).fillna(1.0)

fig = px.bar(g.reset_index(), x="reviewer_id", y="mean",
             title="Reviewer Reliability (posterior mean)",
             labels={"mean": "Reliability", "reviewer_id": "Reviewer"})
fig.show()

/var/folders/vk/fz954n1s6nq4j3ww18fb53140000gn/T/ipykernel_16037/4076598924.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# Model Calibration Training

**What this section does:** Trains calibration models to convert raw OCR confidence scores into meaningful probability estimates.

**Two calibration approaches compared:**

**1. Isotonic Regression:**
- **Method:** Non-parametric approach that enforces monotonic relationships
- **Advantage:** Flexible, doesn't assume specific functional form
- **Best for:** When you want the model to learn the optimal curve shape

**2. Platt Scaling (Logistic Regression):**
- **Method:** Parametric approach using logistic regression
- **Advantage:** Simple, interpretable, fast to compute
- **Best for:** When you want a smooth, well-behaved calibration curve

**Training process:**
- Both models are trained using reviewer reliability weights from the previous section
- Performance is evaluated on validation data using Brier Score and Log Loss
- The better-performing model is selected as the final calibrator

**Output:** A calibration function `best_f(score)` that converts any OCR score to a calibrated probability

**Why calibration matters:**
- **Meaningful probabilities:** Raw scores are arbitrary; calibration makes them interpretable
- **Consistent thresholds:** Enables reliable decision-making across different score ranges
- **Temporal stability:** Calibrated probabilities remain meaningful over time


In [12]:
# --- Isotonic ---
iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(train["score"], train["is_correct"], sample_weight=train["w"])
def f_iso(s): return iso.predict(np.asarray(s, dtype=float))

# --- Platt Scaling ---
Xtr = (train["score"].to_numpy().reshape(-1,1) / 99.0)
lr = LogisticRegression(max_iter=1000)
lr.fit(Xtr, train["is_correct"], sample_weight=train["w"])
def f_platt(s):
    s_arr = np.asarray(s, dtype=float).reshape(-1,1)/99.0
    return lr.predict_proba(s_arr)[:,1]

# --- Evaluate both on validation set ---
p_iso = np.clip(f_iso(val["score"]), 1e-6, 1-1e-6)
p_platt = np.clip(f_platt(val["score"]), 1e-6, 1-1e-6)
m_iso = {"brier": brier_score_loss(val["is_correct"], p_iso),
         "logloss": log_loss(val["is_correct"], p_iso)}
m_platt = {"brier": brier_score_loss(val["is_correct"], p_platt),
           "logloss": log_loss(val["is_correct"], p_platt)}

m_iso, m_platt

({'brier': 0.052272841770937294, 'logloss': 0.16309731165238128},
 {'brier': 0.052044098926547815, 'logloss': 0.1683710552926537})

# Calibration Reliability Check

**What this section does:** Creates a reliability plot to validate how well your calibration model performs by comparing predicted probabilities with observed accuracy rates.

**How to interpret the plot:**
- **X-axis:** Predicted probability of correctness (from your calibration model)
- **Y-axis:** Observed accuracy rate (actual performance in that probability bin)
- **Diagonal line (y=x):** Represents perfect calibration

**Reading the results:**
- **Points on the diagonal:** Perfect calibration - predictions match reality
- **Points below the diagonal:** Model is overconfident - predicting higher probabilities than actual performance
- **Points above the diagonal:** Model is underconfident - predicting lower probabilities than actual performance

**What to look for:**
- **Good calibration:** Points should cluster close to the diagonal line
- **Consistent behavior:** The relationship should be roughly linear
- **Bubble size:** Larger bubbles indicate more data points in that probability range

**Why this matters:**
- **Trust validation:** Ensures your probability estimates are reliable for decision-making
- **Model debugging:** Identifies systematic biases in your calibration
- **Deployment confidence:** Confirms your model is ready for production use


In [13]:
best_f = f_iso if m_iso["brier"] <= m_platt["brier"] else f_platt
best_name = "isotonic" if best_f is f_iso else "platt"

bins = np.arange(0, 101, 5)
val["p_hat"] = best_f(val["score"])
cal_curve = val.groupby(pd.cut(val["score"], bins), observed=False).agg(
    p=("p_hat","mean"),
    y=("is_correct","mean"),
    n=("is_correct","size")
).reset_index()

fig = px.scatter(cal_curve, x="p", y="y", size="n",
                 title=f"Reliability Plot (best={best_name})",
                 labels={"p": "Predicted P(correct)", "y": "Observed Accuracy"})
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Perfect Calibration", line=dict(dash="dot")))
fig.show()

# Threshold Optimization

**What this section does:** Finds the optimal confidence threshold (τ) that balances accuracy and human review workload using a utility function approach.

**The optimization process:**
- **Tests all thresholds:** Evaluates every possible threshold from 0 to 99
- **Calculates key metrics for each threshold:**
  - **Accuracy:** Percentage of correct decisions (accept if score ≥ τ, otherwise review)
  - **Review rate:** Percentage of items sent to human reviewers (score < τ)
  - **Utility:** Accuracy − λ × Review Rate (where λ = cost penalty for human reviews)

**Understanding the utility function:**
- **λ (lambda):** Represents the cost penalty for human reviews
- **Higher λ:** Human reviews are expensive → prefer higher thresholds (trust machine more)
- **Lower λ:** Human reviews are cheap → prefer lower thresholds (send more to humans)
- **Optimal τ:** The threshold that maximizes utility for your chosen λ

**Business implications:**
- **Cost optimization:** Balances accuracy gains against human review costs
- **Workload management:** Controls how much work goes to human reviewers
- **Risk tolerance:** Higher thresholds = more automation, lower thresholds = more human oversight

**Output:** A table showing metrics for all thresholds and the optimal threshold for your cost structure


In [14]:
def choose_threshold(scores, labels, lam=0.2):
    thresholds = np.arange(0, 100)
    best = None
    results = []
    for t in thresholds:
        preds = (scores >= t).astype(int)
        acc = (preds == labels).mean()
        review_rate = (scores < t).mean()
        U = acc - lam * review_rate
        results.append({"tau": t, "accuracy": acc, "review_rate": review_rate, "utility": U})
        if best is None or U > best["utility"]:
            best = {"tau": t, "accuracy": acc, "review_rate": review_rate, "utility": U}
    return pd.DataFrame(results), best

results_df, best = choose_threshold(val["score"].to_numpy(), val["is_correct"].to_numpy(), lam=0.2)
best

{'tau': 78,
 'accuracy': 0.929,
 'review_rate': 0.792,
 'utility': 0.7706000000000001}

# Threshold Trade-off Visualization

**What this section does:** Creates interactive visualizations to help you understand the trade-offs between different threshold choices and their impact on system performance.

**Two key visualizations:**

**1. Utility vs Threshold Plot:**
- **Purpose:** Shows the utility curve to identify the optimal threshold
- **Peak location:** The highest point on the curve represents the best threshold for your cost structure
- **Shape insights:** Steep curves indicate sensitive thresholds, flat curves suggest multiple good options

**2. Accuracy & Review Rate vs Threshold Plot:**
- **Dual metrics:** Shows how accuracy and human review workload change together
- **Trade-off visualization:** Helps you see the relationship between automation and accuracy
- **Threshold impact:** Demonstrates how moving the threshold affects both metrics

**Alternative threshold approaches:**
- **Utility-optimal threshold (dashed line):** Best balance of accuracy and cost for your λ value
- **Probability-based cutoff (dotted line):** Conservative approach using calibrated probabilities (e.g., P(correct) ≥ 0.95)

**How to use these plots:**
- **Peak identification:** Find the threshold that maximizes utility
- **Sensitivity analysis:** See how much utility changes with small threshold adjustments
- **Risk assessment:** Compare utility-optimal vs. probability-based approaches
- **Stakeholder communication:** Use visualizations to explain trade-offs to business stakeholders


In [15]:
# Probability-based threshold (for comparison)
score_grid = np.arange(0, 100)
p_grid = best_f(score_grid)
target = 0.95
candidates = score_grid[p_grid >= target]
tau_prob = int(candidates.min()) if len(candidates) else 99

# --- Plot 1: Utility vs Threshold ---
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=results_df["tau"], y=results_df["utility"],
                          mode="lines", name="Utility"))
fig1.add_vline(x=best["tau"], line_dash="dash", line_color="blue",
               annotation_text=f"Best τ={best['tau']}")
fig1.update_layout(title="Utility vs Threshold",
                   xaxis_title="Threshold (score)", yaxis_title="Utility")
fig1.show()

# --- Plot 2: Accuracy & Review Rate ---
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=results_df["tau"], y=results_df["accuracy"], mode="lines", name="Accuracy"))
fig2.add_trace(go.Scatter(x=results_df["tau"], y=results_df["review_rate"], mode="lines", name="Review Rate"))
fig2.add_vline(x=best["tau"], line_dash="dash", line_color="blue",
               annotation_text=f"Best τ={best['tau']}")
fig2.add_vline(x=tau_prob, line_dash="dot", line_color="orange",
               annotation_text=f"Prob-cut τ={tau_prob} (P≥{target})")
fig2.update_layout(title="Accuracy & Review Rate vs Threshold",
                   xaxis_title="Threshold (score)", yaxis_title="Metric value",
                   xaxis=dict(range=[0, 100]))
fig2.show()

# Model Artifact Persistence

**What this section does:** Saves the trained calibration model and optimal threshold to disk for deployment in production systems.

**Files created:**
- **`calibrator.pkl`:** Contains the trained calibration function that converts OCR scores to probabilities
- **`threshold.json`:** Stores the optimal threshold value and calibration method used

**Artifact contents:**
- **Calibration function:** The `best_f(score)` function that maps any OCR score to a calibrated probability
- **Optimal threshold:** The τ value that maximizes utility for your cost structure
- **Model metadata:** Information about which calibration method (isotonic vs. platt) was selected

**Production deployment benefits:**
- **Consistent decisions:** Your application can load these artifacts and make identical decisions
- **No retraining needed:** Deploy immediately without rebuilding models
- **Version control:** Track different model versions and rollback if needed
- **Performance:** Pre-trained models load quickly for real-time inference

**Usage in production:**
```python
# Load artifacts
with open('artifacts/calibrator.pkl', 'rb') as f:
    calibrator = pickle.load(f)
with open('artifacts/threshold.json', 'r') as f:
    config = json.load(f)

# Make decisions
score = 85  # OCR confidence score
prob = calibrator(score)  # Calibrated probability
decision = "accept" if score >= config['tau'] else "review"
```


In [16]:
import pickle, json, pathlib

art = pathlib.Path("artifacts"); art.mkdir(exist_ok=True)
with open(art/"calibrator.pkl","wb") as f:
    pickle.dump(best_f, f)
with open(art/"threshold.json","w") as f:
    json.dump({"tau": int(best["tau"]), "type": best_name}, f)

print("✅ Saved calibrator and threshold configuration.")

✅ Saved calibrator and threshold configuration.


# Production Validation with Drift Simulation

**What this section does:** Tests your deployed model on new data to ensure it still performs well, even when the underlying data distribution may have changed (model drift).

**Validation process:**
- **New data simulation:** Generates a fresh batch of OCR data (optionally with drift parameters)
- **Performance comparison:** Evaluates three different threshold strategies:
  - **Saved threshold:** Your previously optimized τ from training
  - **Optimal threshold:** The best τ for this new batch (ground truth)
  - **Probability cutoff:** Conservative approach using calibrated probabilities

**Drift simulation options:**
- **`delta_tau`:** Shifts the true threshold (e.g., -3 means threshold moved down by 3 points)
- **`delta_slope`:** Changes the transition sharpness (e.g., +1.0 makes the curve steeper)
- **Real-world scenarios:** Simulates how OCR system changes, data quality shifts, or domain changes affect performance

**Key metrics reported:**
- **Utility loss:** Difference between saved threshold performance and optimal performance
- **Accuracy comparison:** How well each threshold strategy performs
- **Review rate impact:** How much human workload each approach requires

**What to look for:**
- **Low utility loss:** Your saved threshold should perform close to the optimal threshold
- **Stable performance:** Small changes in utility indicate robust model deployment
- **Drift sensitivity:** How much performance degrades when data distribution changes

**Production monitoring:** Use this approach to continuously validate your deployed model and detect when retraining is needed.


In [17]:
# Cell 10 — Validate Saved Calibrator/Threshold on New Data (with optional drift)

import numpy as np
import pandas as pd
import json, pickle, pathlib
import plotly.graph_objects as go

# -----------------------------
# 0) Config
# -----------------------------
NEW_N = 5000
LAM   = 0.2           # same λ used for utility during training
DRIFT = {"delta_tau": 0, "delta_slope": 0.0}  # e.g., {"delta_tau": -3, "delta_slope": +1.0}

# -----------------------------
# 1) Load artifacts
# -----------------------------
art = pathlib.Path("artifacts")
with open(art/"calibrator.pkl","rb") as f:
    loaded_calibrator = pickle.load(f)
with open(art/"threshold.json","r") as f:
    saved = json.load(f)
TAU_SAVED = int(saved["tau"])
CAL_TYPE  = saved.get("type", "unknown")

print(f"Loaded calibrator type: {CAL_TYPE}, saved τ={TAU_SAVED}")

# -----------------------------
# 2) Simulate a new batch (optionally with drift)
# -----------------------------
rng = np.random.default_rng(2025)

# Reuse TRUE_TAU and K from earlier cells if they exist; else define defaults
try:
    TRUE_TAU
except NameError:
    TRUE_TAU = 80
try:
    K
except NameError:
    K = 5.0

tau_new = TRUE_TAU + DRIFT["delta_tau"]
k_new   = K + DRIFT["delta_slope"]

def p_correct_new(score, tau=tau_new, k=k_new):
    return 1.0 / (1.0 + np.exp(-(score - tau) / k))

scores_new = rng.integers(0, 100, size=NEW_N)
is_correct_new = rng.binomial(1, p_correct_new(scores_new))

df_new = pd.DataFrame({
    "id": np.arange(NEW_N),
    "score": scores_new,
    "is_correct": is_correct_new
}).sort_values("id").reset_index(drop=True)

# -----------------------------
# 3) Evaluate saved threshold on new batch
# -----------------------------
s = df_new["score"].to_numpy()
y = df_new["is_correct"].to_numpy()

def eval_at_tau(scores, labels, tau, lam=LAM):
    preds = (scores >= tau).astype(int)
    acc = (preds == labels).mean()
    review = (scores < tau).mean()
    util = acc - lam * review
    return acc, review, util

acc_saved, review_saved, util_saved = eval_at_tau(s, y, TAU_SAVED, lam=LAM)

# -----------------------------
# 4) Find ground-truth optimal τ on new batch
# -----------------------------
taus = np.arange(0, 100)
utils = []
for t in taus:
    _, _, u = eval_at_tau(s, y, t, lam=LAM)
    utils.append(u)
utils = np.array(utils)
best_idx = int(np.argmax(utils))
TAU_TRUE_STAR = int(taus[best_idx])
ACC_TRUE_STAR, REVIEW_TRUE_STAR, UTIL_TRUE_STAR = eval_at_tau(s, y, TAU_TRUE_STAR, lam=LAM)

# -----------------------------
# 5) Probability-based cutoff using loaded calibrator (optional)
# -----------------------------
# The saved calibrator maps score -> P(correct); we use it to find prob-cutoff score for P ≥ 0.95
try:
    p_grid = np.asarray(loaded_calibrator(np.arange(0,100)), dtype=float)
    target = 0.95
    cands = np.arange(0,100)[p_grid >= target]
    TAU_PROB = int(cands.min()) if len(cands) else 99
except Exception:
    p_grid, TAU_PROB, target = None, None, 0.95

# -----------------------------
# 6) Plotly visuals (Utility curve + markers)
# -----------------------------
fig = go.Figure()
fig.add_trace(go.Scatter(x=taus, y=utils, mode="lines", name="Utility"))
fig.add_vline(x=TAU_SAVED, line_dash="dash", line_color="blue",
              annotation_text=f"Saved τ={TAU_SAVED}")
fig.add_vline(x=TAU_TRUE_STAR, line_dash="dot", line_color="green",
              annotation_text=f"Best τ on new data={TAU_TRUE_STAR}")
if TAU_PROB is not None:
    fig.add_vline(x=TAU_PROB, line_dash="dashdot", line_color="orange",
                  annotation_text=f"Prob-cut τ={TAU_PROB} (P≥{target:.2f})")
fig.update_layout(title="Utility vs Threshold (New Batch)",
                  xaxis_title="Threshold (score)",
                  yaxis_title="Utility",
                  hovermode="x unified")
fig.show()

# -----------------------------
# 7) Secondary plot: Accuracy & Review rate vs τ (optional)
# -----------------------------
accs = []; reviews = []
for t in taus:
    a, r, _ = eval_at_tau(s, y, t, lam=LAM)
    accs.append(a); reviews.append(r)

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=taus, y=accs, mode="lines", name="Accuracy"))
fig2.add_trace(go.Scatter(x=taus, y=reviews, mode="lines", name="Review rate"))
fig2.add_vline(x=TAU_SAVED, line_dash="dash", line_color="blue",
               annotation_text=f"Saved τ={TAU_SAVED}")
fig2.add_vline(x=TAU_TRUE_STAR, line_dash="dot", line_color="green",
               annotation_text=f"Best τ={TAU_TRUE_STAR}")
if TAU_PROB is not None:
    fig2.add_vline(x=TAU_PROB, line_dash="dashdot", line_color="orange",
                   annotation_text=f"Prob-cut τ={TAU_PROB} (P≥{target:.2f})")
fig2.update_layout(title="Accuracy & Review Rate vs Threshold (New Batch)",
                   xaxis_title="Threshold (score)",
                   yaxis_title="Metric value",
                   hovermode="x unified")
fig2.show()

# -----------------------------
# 8) Print concise summary
# -----------------------------
util_loss = UTIL_TRUE_STAR - util_saved
pct_loss = 0.0 if UTIL_TRUE_STAR == 0 else (util_loss / abs(UTIL_TRUE_STAR)) * 100.0

print("=== New Batch Evaluation ===")
print(f"Saved τ: {TAU_SAVED} | Acc={acc_saved:.3f} | Review={review_saved:.3f} | Utility={util_saved:.3f}")
print(f"Best τ (new data): {TAU_TRUE_STAR} | Acc={ACC_TRUE_STAR:.3f} | Review={REVIEW_TRUE_STAR:.3f} | Utility={UTIL_TRUE_STAR:.3f}")
if TAU_PROB is not None:
    acc_prob, review_prob, util_prob = eval_at_tau(s, y, TAU_PROB, lam=LAM)
    print(f"Prob-cut τ (P≥{target:.2f}): {TAU_PROB} | Acc={acc_prob:.3f} | Review={review_prob:.3f} | Utility={util_prob:.3f}")
print(f"Utility loss of saved τ vs best: {util_loss:.3f} ({pct_loss:.2f}%)")
print(f"Drift simulated: Δτ={DRIFT['delta_tau']}, Δslope={DRIFT['delta_slope']}")

Loaded calibrator type: platt, saved τ=78


=== New Batch Evaluation ===
Saved τ: 78 | Acc=0.936 | Review=0.786 | Utility=0.778
Best τ (new data): 79 | Acc=0.939 | Review=0.795 | Utility=0.780
Prob-cut τ (P≥0.95): 99 | Acc=0.816 | Review=0.988 | Utility=0.618
Utility loss of saved τ vs best: 0.002 (0.19%)
Drift simulated: Δτ=0, Δslope=0.0


In [18]:
# -----------------------------
# 6) Plotly visuals (Utility curve + markers)
# -----------------------------
fig = go.Figure()
fig.add_trace(go.Scatter(x=taus, y=utils, mode="lines", name="Utility"))
fig.add_vline(x=TAU_SAVED, line_dash="dash", line_color="blue")
fig.add_vline(x=TAU_TRUE_STAR, line_dash="dot", line_color="green")
if TAU_PROB is not None:
    fig.add_vline(x=TAU_PROB, line_dash="dashdot", line_color="orange")

# Add non-overlapping annotations separately
annots = [
    dict(x=TAU_SAVED, y=utils.max(), text=f"Saved τ={TAU_SAVED}",
         showarrow=True, arrowhead=2, yshift=20, ax=40, ay=-40, font=dict(size=12)),
    dict(x=TAU_TRUE_STAR, y=utils.max()*0.95, text=f"Best τ={TAU_TRUE_STAR}",
         showarrow=True, arrowhead=2, yshift=20, ax=-40, ay=-40, font=dict(size=12))
]
if TAU_PROB is not None:
    annots.append(
        dict(x=TAU_PROB, y=utils.max()*0.9, text=f"Prob-cut τ={TAU_PROB} (P≥{target:.2f})",
             showarrow=True, arrowhead=2, yshift=20, ax=40, ay=-40, font=dict(size=12))
    )

fig.update_layout(
    title="Utility vs Threshold (New Batch)",
    xaxis_title="Threshold (score)",
    yaxis_title="Utility",
    annotations=annots,
    hovermode="x unified",
    margin=dict(l=60, r=40, t=60, b=50),
    template="plotly_white"
)
fig.show()

# -----------------------------
# 7) Secondary plot: Accuracy & Review rate vs τ
# -----------------------------
accs = []; reviews = []
for t in taus:
    a, r, _ = eval_at_tau(s, y, t, lam=LAM)
    accs.append(a); reviews.append(r)

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=taus, y=accs, mode="lines", name="Accuracy"))
fig2.add_trace(go.Scatter(x=taus, y=reviews, mode="lines", name="Review rate"))
fig2.add_vline(x=TAU_SAVED, line_dash="dash", line_color="blue")
fig2.add_vline(x=TAU_TRUE_STAR, line_dash="dot", line_color="green")
if TAU_PROB is not None:
    fig2.add_vline(x=TAU_PROB, line_dash="dashdot", line_color="orange")

fig2.update_layout(
    title="Accuracy & Review Rate vs Threshold (New Batch)",
    xaxis_title="Threshold (score)",
    yaxis_title="Metric value",
    hovermode="x unified",
    margin=dict(l=60, r=40, t=60, b=50),
    template="plotly_white",
    annotations=[
        dict(x=TAU_SAVED, y=max(accs), text=f"Saved τ={TAU_SAVED}", showarrow=False, yshift=25, font=dict(size=12)),
        dict(x=TAU_TRUE_STAR, y=max(accs)*0.95, text=f"Best τ={TAU_TRUE_STAR}", showarrow=False, yshift=25, font=dict(size=12)),
        dict(x=TAU_PROB, y=max(accs)*0.9, text=f"Prob-cut τ={TAU_PROB} (P≥{target:.2f})",
             showarrow=False, yshift=25, font=dict(size=12)) if TAU_PROB is not None else {}
    ]
)
fig2.show()

# Baseline Configuration — OCR Threshold Optimization

### Experiment Metadata
| Setting | Value | Notes |
|----------|--------|-------|
| Date | 2025-10-21 | Baseline run |
| Version | v1.0 | Increment when re-running pipeline |
| Dataset size | n = 5000 | Simulated OCR batch |
| Drift config | Δτ = 0, Δslope = 0.0 | Used for new batch test |
| Train / Validation split | 80 / 20 | Time-based split |
| λ (review-cost weight) | 0.2 | Penalty term in utility function |


### Model Configuration
| Component | Setting | Description |
|------------|----------|-------------|
| Calibration algorithm | Isotonic Regression | Maps OCR score to calibrated P(correct) |
| Alternative model tested | Logistic Regression (Platt Scaling) | Simpler sigmoid calibration |
| Reviewer weighting | Beta-Bernoulli posterior mean | Weights reviewers by reliability |
| Threshold optimization | Utility = Accuracy – λ × ReviewRate | Searches thresholds 0–99 |
| Probability cutoff (optional) | P(correct) ≥ 0.95 | Defines stricter "auto-accept" mode |


### Results Summary
| Metric | Value | Meaning |
|---------|--------|---------|
| Best τ (utility-optimal) | 78 | Learned decision threshold |
| Accuracy at τ | ≈ 0.93 | Overall system accuracy |
| Review rate | ≈ 0.79 | Percentage routed to human review |
| Utility | ≈ 0.77 | Combined accuracy–cost score |
| Probability-cut τ | 99 | Risk-averse acceptance cutoff |
| Calibrator type used | Isotonic Regression | Chosen by lowest Brier/LogLoss |
| Reviewer reliabilities | A ≈ 0.95, B ≈ 0.85, C ≈ 0.70 | Estimated from audits |
| Validation utility loss | 0% | Perfect generalization on new batch |


### Artifacts
| File | Description |
|------|--------------|
| artifacts/calibrator.pkl | Saved calibration model |
| artifacts/threshold.json | Saved τ configuration |
| plots/utility_vs_threshold.html | Utility optimization visualization |
| plots/reliability_curve.html | Calibration reliability visualization |


### Next Steps
- Freeze this configuration as baseline v1.0
- Implement online updating (rolling 30-day recalibration)
- Add drift monitoring for τ, utility, and calibration loss
- Track reviewer reliability over time
- Run λ-sensitivity analysis (λ = 0.1 – 0.5)

In [19]:
# Requirements (once):  !pip -q install numpy pandas scikit-learn plotly

import json, pickle, pathlib
from datetime import timedelta

import numpy as np
import pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss

ART = pathlib.Path("artifacts"); ART.mkdir(exist_ok=True)
LOG = pathlib.Path("logs"); LOG.mkdir(exist_ok=True)

# ---------- Reviewer weights (Beta-Bernoulli with light shrinkage) ----------
def reviewer_weights(df, alpha0=1.0, beta0=1.0, min_n=10, shrink=0.5):
    # df needs: reviewer_id, review_label, is_correct
    g = df.groupby("reviewer_id").apply(
        lambda d: pd.Series({
            "n": len(d),
            "agree": (d["review_label"] == d["is_correct"]).sum()
        })
    )
    g["alpha"] = alpha0 + g["agree"]
    g["beta"]  = beta0 + (g["n"] - g["agree"])
    mean = g["alpha"] / (g["alpha"] + g["beta"])
    # shrink toward 0.5 for low-volume reviewers
    w = {}
    for rid, m, n in zip(mean.index, mean.values, g["n"].values):
        w[rid] = (shrink * m + (1 - shrink) * 0.5) if n < min_n else float(m)
    return w

# ---------- Fit calibrators; return best by Brier/LogLoss on a val split ----------
def fit_best_calibrator(df_train: pd.DataFrame, df_val: pd.DataFrame, weights: pd.Series):
    # Isotonic
    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(df_train["score"].astype(float), df_train["is_correct"].astype(float), sample_weight=weights)
    def f_iso(x): 
        x = np.asarray(x, dtype=float)
        return iso.predict(x)

    # Platt (logistic)
    Xtr = (df_train["score"].to_numpy().reshape(-1,1) / 99.0)
    lr = LogisticRegression(max_iter=1000)
    lr.fit(Xtr, df_train["is_correct"], sample_weight=weights)
    def f_platt(x):
        x = np.asarray(x, dtype=float).reshape(-1,1)/99.0
        return lr.predict_proba(x)[:,1]

    # Evaluate both
    def eval_f(f, df):
        p = np.clip(f(df["score"]), 1e-6, 1-1e-6)
        return dict(brier=brier_score_loss(df["is_correct"], p), logloss=log_loss(df["is_correct"], p))
    m_iso   = eval_f(f_iso, df_val)
    m_platt = eval_f(f_platt, df_val)

    if (m_iso["brier"], m_iso["logloss"]) <= (m_platt["brier"], m_platt["logloss"]):
        return {"name": "isotonic", "model": iso, "predict": f_iso, "metrics": m_iso}
    else:
        return {"name": "platt", "model": lr, "predict": f_platt, "metrics": m_platt}

# ---------- Utility over thresholds ----------
def utility_curve(scores, labels, lam=0.2):
    taus = np.arange(0, 100)
    s = np.asarray(scores)
    y = np.asarray(labels)
    accs, reviews, utils = [], [], []
    for t in taus:
        preds = (s >= t).astype(int)
        acc = (preds == y).mean()
        review = (s < t).mean()
        U = acc - lam * review
        accs.append(acc); reviews.append(review); utils.append(U)
    arr = pd.DataFrame({"tau": taus, "accuracy": accs, "review_rate": reviews, "utility": utils})
    best_row = arr.iloc[arr["utility"].idxmax()]
    return arr, best_row

# ---------- Save/load artifacts ----------
def save_artifacts(calibrator_name, calibrator_model, tau, meta):
    # Save calibrator
    with open(ART/"calibrator.pkl","wb") as f:
        pickle.dump({"type": calibrator_name, "model": calibrator_model}, f)
    # Save threshold and meta
    out = {"tau": int(tau), "type": calibrator_name}
    out.update(meta or {})
    with open(ART/"threshold.json","w") as f:
        json.dump(out, f, indent=2)

def load_artifacts():
    with open(ART/"calibrator.pkl","rb") as f:
        cal = pickle.load(f)
    with open(ART/"threshold.json","r") as f:
        thr = json.load(f)
    return cal, thr

In [20]:
def update_threshold_online(
    df_all: pd.DataFrame,
    window_days: int = 30,
    lam: float = 0.2,
    min_records: int = 1000,
    drift_cap: int = 3,          # max allowed change in tau per update; None to disable
    log_file: str = "logs/updates.csv"
):
    """
    Refit calibrator and recompute threshold using the most recent `window_days` of data.
    Expects df_all to include columns:
      ['id','score','is_correct','reviewer_id','review_label','timestamp']
    Returns a dict summary and logs to CSV.
    """
    # Ensure time sorted
    df_all = df_all.copy()
    df_all["timestamp"] = pd.to_datetime(df_all["timestamp"])
    df_all.sort_values("timestamp", inplace=True)

    t_max = df_all["timestamp"].max()
    cutoff = t_max - timedelta(days=window_days)
    win = df_all[df_all["timestamp"] >= cutoff].copy()
    if len(win) < min_records:
        return {"status": "skipped", "reason": f"not_enough_records ({len(win)} < {min_records})"}

    # Train/val split within window (time-based 80/20)
    cut = int(0.8 * len(win))
    train = win.iloc[:cut].copy()
    val   = win.iloc[cut:].copy()

    # Reviewer weights
    w_map = reviewer_weights(train)
    train["w"] = train["reviewer_id"].map(w_map).fillna(1.0)

    # Fit calibrator (pick best)
    best = fit_best_calibrator(train, val, weights=train["w"])
    f = best["predict"]

    # Choose threshold via utility
    curve, best_row = utility_curve(val["score"], val["is_correct"], lam=lam)
    tau_star = int(best_row["tau"])

    # Optional drift cap vs previous tau
    prev_tau = None
    try:
        _, thr = load_artifacts()
        prev_tau = int(thr["tau"])
        if drift_cap is not None and prev_tau is not None:
            if abs(tau_star - prev_tau) > drift_cap:
                tau_star = prev_tau + int(np.sign(tau_star - prev_tau) * drift_cap)
    except Exception:
        pass  # first run, no artifacts yet

    # Compute calibration losses on val
    p_val = np.clip(f(val["score"]), 1e-6, 1-1e-6)
    brier = brier_score_loss(val["is_correct"], p_val)
    lgls  = log_loss(val["is_correct"], p_val)

    # Persist artifacts
    meta = {
        "window_days": window_days,
        "lam": lam,
        "val_size": int(len(val)),
        "brier": float(brier),
        "logloss": float(lgls),
        "timestamp_max": t_max.isoformat()
    }
    save_artifacts(best["name"], best["model"], tau_star, meta)

    # Log update row
    log_path = pathlib.Path(log_file)
    rec = {
        "timestamp_max": t_max,
        "window_days": window_days,
        "lam": lam,
        "tau": tau_star,
        "prev_tau": prev_tau,
        "brier": brier,
        "logloss": lgls,
        "val_size": len(val),
        "calibrator": best["name"]
    }
    if log_path.exists():
        pd.DataFrame([rec]).to_csv(log_path, mode="a", header=False, index=False)
    else:
        pd.DataFrame([rec]).to_csv(log_path, index=False)

    return {"status": "ok", **rec, "curve": curve}

In [21]:
import plotly.express as px
import plotly.graph_objects as go

def monitor_updates(log_file: str = "logs/updates.csv"):
    log_path = pathlib.Path(log_file)
    if not log_path.exists():
        raise FileNotFoundError(f"No log file found at {log_path}")

    dfu = pd.read_csv(log_path, parse_dates=["timestamp_max"])
    if dfu.empty:
        raise ValueError("Log is empty.")

    # Tau over time
    fig1 = px.line(dfu, x="timestamp_max", y="tau", title="Threshold τ over time",
                   markers=True, color_discrete_sequence=["#1f77b4"])
    if "prev_tau" in dfu.columns and dfu["prev_tau"].notna().any():
        fig1.add_trace(go.Scatter(x=dfu["timestamp_max"], y=dfu["prev_tau"],
                                  mode="lines+markers", name="prev_tau",
                                  line=dict(dash="dot", color="#888")))

    fig1.update_layout(xaxis_title="Time", yaxis_title="τ", template="plotly_white")
    fig1.show()

    # Calibration loss over time
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=dfu["timestamp_max"], y=dfu["brier"], mode="lines+markers", name="Brier"))
    fig2.add_trace(go.Scatter(x=dfu["timestamp_max"], y=dfu["logloss"], mode="lines+markers", name="LogLoss"))
    fig2.update_layout(title="Calibration losses over time",
                       xaxis_title="Time", yaxis_title="Loss",
                       template="plotly_white")
    fig2.show()

    # If you logged utility curves in updates, you could add a utility-over-time plot here.

In [22]:
# Assume df is your cumulative dataset with the required columns
# Run an update on the latest 30 days and log it
summary = update_threshold_online(df, window_days=30, lam=0.2, min_records=1000, drift_cap=3)
summary if isinstance(summary, dict) else None

/var/folders/vk/fz954n1s6nq4j3ww18fb53140000gn/T/ipykernel_16037/2224041227.py:18: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



{'status': 'ok',
 'timestamp_max': Timestamp('2025-09-29 00:00:00'),
 'window_days': 30,
 'lam': 0.2,
 'tau': 76,
 'prev_tau': 78,
 'brier': 0.05475335321774615,
 'logloss': 0.17978138081540074,
 'val_size': 526,
 'calibrator': 'platt',
 'curve':     tau  accuracy  review_rate   utility
 0     0  0.211027     0.000000  0.211027
 1     1  0.214829     0.003802  0.214068
 2     2  0.216730     0.005703  0.215589
 3     3  0.224335     0.013308  0.221673
 4     4  0.231939     0.020913  0.227757
 ..  ...       ...          ...       ...
 95   95  0.847909     0.937262  0.660456
 96   96  0.844106     0.944867  0.655133
 97   97  0.825095     0.963878  0.632319
 98   98  0.815589     0.973384  0.620913
 99   99  0.804183     0.984791  0.607224
 
 [100 rows x 4 columns]}

In [23]:
# Visualize how τ and calibration losses evolve across updates
monitor_updates("logs/updates.csv")

In [24]:
# Plotly dashboard: Historical drift of τ, Brier, and LogLoss
# -----------------------------------------------------------
# Assumes your updater logs: logs/updates.csv with columns:
#   timestamp_max, tau, prev_tau, brier, logloss, val_size, calibrator
#
# Optional: pass a DataFrame truth_df with columns:
#   timestamp_max, true_tau
#
# Example:
#   dashboard = render_drift_dashboard("logs/updates.csv", truth_df=truth_df)
#   dashboard.show()

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

def render_drift_dashboard(log_file: str = "logs/updates.csv", truth_df: pd.DataFrame | None = None):
    p = Path(log_file)
    if not p.exists():
        raise FileNotFoundError(f"Log file not found: {p}")

    dfu = pd.read_csv(p, parse_dates=["timestamp_max"])
    if dfu.empty:
        raise ValueError("Log file is empty.")

    # Sort and basic cleaning
    dfu = dfu.sort_values("timestamp_max").reset_index(drop=True)
    # Optional: rolling metrics for smoother trend lines (tune window as needed)
    dfu["tau_roll"]   = dfu["tau"].rolling(3, min_periods=1).mean()
    dfu["brier_roll"] = dfu["brier"].rolling(3, min_periods=1).mean()
    dfu["logloss_roll"] = dfu["logloss"].rolling(3, min_periods=1).mean()

    # Prepare truth overlay if provided
    if truth_df is not None and not truth_df.empty:
        truth = truth_df.copy()
        if "timestamp_max" not in truth.columns or "true_tau" not in truth.columns:
            raise ValueError("truth_df must have columns: ['timestamp_max','true_tau']")
        truth["timestamp_max"] = pd.to_datetime(truth["timestamp_max"])
        truth = truth.sort_values("timestamp_max").reset_index(drop=True)
    else:
        truth = None

    # Build subplots: τ on top, Brier & LogLoss on bottom (shared x, separate y)
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.12,
        row_heights=[0.5, 0.5],
        subplot_titles=("Threshold τ over time", "Calibration losses (Brier & LogLoss) over time")
    )

    # --- Row 1: τ over time ---
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["tau"], mode="lines+markers",
            name="τ (learned)", line=dict(width=2)
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["tau_roll"], mode="lines",
            name="τ (3-pt rolling)", line=dict(width=2, dash="dot")
        ),
        row=1, col=1
    )
    if "prev_tau" in dfu.columns and dfu["prev_tau"].notna().any():
        fig.add_trace(
            go.Scatter(
                x=dfu["timestamp_max"], y=dfu["prev_tau"], mode="lines+markers",
                name="prev τ", line=dict(width=1, dash="dash"), marker=dict(size=6, symbol="circle-open")
            ),
            row=1, col=1
        )
    if truth is not None:
        fig.add_trace(
            go.Scatter(
                x=truth["timestamp_max"], y=truth["true_tau"], mode="lines+markers",
                name="true τ (simulated)", line=dict(width=2, dash="dashdot")
            ),
            row=1, col=1
        )

    # --- Row 2: Brier & LogLoss over time ---
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["brier"], mode="lines+markers",
            name="Brier", line=dict(width=2), yaxis="y2"
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["brier_roll"], mode="lines",
            name="Brier (3-pt rolling)", line=dict(width=2, dash="dot"), yaxis="y2"
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["logloss"], mode="lines+markers",
            name="LogLoss", line=dict(width=2)
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=dfu["timestamp_max"], y=dfu["logloss_roll"], mode="lines",
            name="LogLoss (3-pt rolling)", line=dict(width=2, dash="dot")
        ),
        row=2, col=1
    )

    # Layout & cosmetics
    fig.update_layout(
        title="Historical Drift Dashboard",
        hovermode="x unified",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
        margin=dict(l=60, r=30, t=70, b=50),
        xaxis=dict(title="Time", rangeslider=dict(visible=True)),
    )
    fig.update_yaxes(title_text="τ", row=1, col=1)
    fig.update_yaxes(title_text="Loss", row=2, col=1)

    # Annotate calibrator type as markers (optional)
    if "calibrator" in dfu.columns:
        # Add small text markers above τ line to show which calibrator was chosen each update.
        for xi, yi, cal in zip(dfu["timestamp_max"], dfu["tau"], dfu["calibrator"]):
            fig.add_annotation(
                x=xi, y=yi, text=str(cal), showarrow=False, yshift=18, font=dict(size=10, color="#666"),
                xanchor="center"
            )

    return fig

In [25]:
# Basic: just what the updater has logged so far
fig = render_drift_dashboard("logs/updates.csv")
fig.show()

In [26]:
# Simulate New OCR Data Function
# --------------------------------
# This function generates new synthetic OCR data for testing online updates

def simulate_batch(start_date='2025-11-21', n=1200, true_tau=75, k=5.0):
    """
    Simulate a new batch of OCR data with specified parameters.
    
    Parameters:
    -----------
    start_date : str or datetime
        Starting date for the new batch
    n : int
        Number of samples to generate
    true_tau : float
        True threshold for the sigmoid function
    k : float
        Slope parameter for the sigmoid function
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: id, score, is_correct, reviewer_id, review_label, timestamp
    """
    rng = np.random.default_rng(42)
    
    # Convert start_date to datetime if it's a string
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    
    def p_correct(score, tau=true_tau, k=k):
        """True underlying probability that OCR is correct."""
        return 1 / (1 + np.exp(-(score - tau) / k))
    
    # Generate new data
    scores = rng.integers(0, 100, size=n)
    probs = p_correct(scores)
    is_correct = rng.binomial(1, probs)
    
    # Use same reviewers as before
    REVIEWERS = {"A": 0.95, "B": 0.85, "C": 0.70}
    reviewer_id = rng.choice(list(REVIEWERS.keys()), size=n)
    review_label = np.array([
        (1 - is_correct[i]) if (rng.random() > REVIEWERS[r]) else is_correct[i]
        for i, r in enumerate(reviewer_id)
    ])
    
    # Generate timestamps starting from start_date
    timestamp = start_date + pd.to_timedelta(
        rng.integers(0, 30, size=n), unit="D"  # Spread over 30 days
    )
    
    # Create new IDs (continue from existing data)
    new_df = pd.DataFrame({
        "id": np.arange(n),  # Will be adjusted when concatenated
        "score": scores,
        "is_correct": is_correct,
        "reviewer_id": reviewer_id,
        "review_label": review_label,
        "timestamp": timestamp
    }).sort_values("timestamp").reset_index(drop=True)
    
    return new_df

print("✅ simulate_batch function defined")


✅ simulate_batch function defined


In [27]:
# Online Threshold Update Example
# --------------------------------
# This demonstrates how to simulate new data and update the threshold online

from datetime import timedelta

# Simulate new OCR data 30 days later
new_batch = simulate_batch(
    start_date='2025-11-21',  # or datetime.now() + timedelta(days=30)
    n=1200,
    true_tau=75,              # simulate a slight model improvement
    k=5.0
)

df_all = pd.concat([df, new_batch], ignore_index=True)

summary = update_threshold_online(
    df_all,
    window_days=30,
    lam=0.2,
    min_records=1000,
    drift_cap=3,
    log_file="logs/updates.csv"
)

print("✅ Online update completed!")
print(f"New threshold: {summary['tau']}")
print(f"Previous threshold: {summary.get('prev_tau', 'N/A')}")
print(f"Brier score: {summary['brier']:.4f}")
print(f"Log loss: {summary['logloss']:.4f}")
print(f"Validation size: {summary['val_size']}")
print(f"Calibrator used: {summary['calibrator']}")


✅ Online update completed!
New threshold: 77
Previous threshold: 76
Brier score: 0.0385
Log loss: 0.1729
Validation size: 240
Calibrator used: isotonic


/var/folders/vk/fz954n1s6nq4j3ww18fb53140000gn/T/ipykernel_16037/2224041227.py:18: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
# Debug: Check if the online update worked
print("Checking CSV file...")
import pandas as pd
try:
    log_df = pd.read_csv("logs/updates.csv")
    print(f"CSV file has {len(log_df)} entries")
    print("Latest entries:")
    print(log_df.tail())
except Exception as e:
    print(f"Error reading CSV: {e}")

# Check if df_all was created properly
try:
    print(f"\ndf_all shape: {df_all.shape}")
    print(f"df_all date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
    print(f"New batch shape: {new_batch.shape}")
except Exception as e:
    print(f"Error with dataframes: {e}")
